In [1]:
import pandas as pd 

In [2]:
def rename_cols(dataframe,building): 
    dataframe.rename(columns = {"kWh": str(building+"_kWh"),"kW": str(building + "_kW")},inplace = True)
    dataframe.head()

In [3]:
def add_name(dataframe,building):
    dataframe['building'] = building

In [4]:
olin = pd.read_csv("olin.csv.csv")
rename_cols(olin,"olin")
olin

,Read Date,olin_kWh,olin_kW
0,1/1/2020 12:00:00 AM,7.0,28.0
1,1/1/2020 12:15:00 AM,6.8,27.2
2,1/1/2020 12:30:00 AM,7.0,28.0
3,1/1/2020 12:45:00 AM,7.0,28.0
4,1/1/2020 1:00:00 AM,7.0,28.0
...,...,...,...
45975,4/25/2021 11:45:00 PM,21.4,85.6
45976,4/26/2021 12:00:00 AM,22.0,88.0
45977,4/26/2021 12:15:00 AM,21.6,86.4
45978,4/26/2021 12:30:00 AM,21.4,85.6


In [5]:
old_centre = pd.read_csv("OldCentre.csv") 
rename_cols(old_centre,"old_centre")
old_centre

,Read Date,old_centre_kWh,old_centre_kW
0,1/1/2020 12:00:00 AM,0.0450,0.1800
1,1/1/2020 12:15:00 AM,0.0410,0.1640
2,1/1/2020 12:30:00 AM,0.0440,0.1760
3,1/1/2020 12:45:00 AM,0.0410,0.1640
4,1/1/2020 1:00:00 AM,0.0429,0.1716
...,...,...,...
45940,4/25/2021 11:45:00 PM,8.0000,32.0000
45941,4/26/2021 12:00:00 AM,7.2000,28.8000
45942,4/26/2021 12:15:00 AM,8.4000,33.6000
45943,4/26/2021 12:30:00 AM,8.4000,33.6000


In [6]:
jvac = pd.read_csv("jvac.csv")
rename_cols(jvac,"jvac")
jvac

,Read Date,jvac_kWh,jvac_kW
0,1/1/2020 0:00,3.2,12.8
1,1/1/2020 0:15,3.4,13.6
2,1/1/2020 0:30,3.6,14.4
3,1/1/2020 0:45,3.6,14.4
4,1/1/2020 1:00,3.6,14.4
...,...,...,...
51111,4/26/2021 0:15,5.2,NaN
51112,4/26/2021 0:30,0.0,NaN
51113,4/26/2021 0:30,5.4,NaN
51114,4/26/2021 0:45,0.0,NaN


In [7]:
crounse = pd.read_csv("crounse.csv")
rename_cols(crounse,"crounse")
crounse

,Read Date,crounse_kWh,crounse_kW
0,1/1/2020 0:00,0.302,1.208
1,1/1/2020 0:15,0.293,1.172
2,1/1/2020 0:30,0.249,0.996
3,1/1/2020 0:45,0.315,1.260
4,1/1/2020 1:00,0.287,1.148
...,...,...,...
45975,4/25/2021 23:45,0.489,1.956
45976,4/26/2021 0:00,0.503,2.012
45977,4/26/2021 0:15,0.525,2.100
45978,4/26/2021 0:30,0.494,1.976


In [8]:
warehouse = pd.read_csv("warehouse.csv")
rename_cols(warehouse,"warehouse")
warehouse

,Read Date,warehouse_kWh,warehouse_kW
0,1/1/2020 0:00,4.80,19.20
1,1/1/2020 0:15,4.92,19.68
2,1/1/2020 0:30,4.80,19.20
3,1/1/2020 0:45,4.92,19.68
4,1/1/2020 1:00,4.80,19.20
...,...,...,...
46071,4/25/2021 23:45,3.36,13.44
46072,4/26/2021 0:00,3.48,13.92
46073,4/26/2021 0:15,3.60,14.40
46074,4/26/2021 0:30,3.24,12.96


In [9]:
joined= pd.concat([crounse,jvac,old_centre])

In [10]:
joined = pd.merge(crounse,warehouse)
add_j = pd.merge(jvac,joined,how="inner",left_index=True)
new = pd.merge(add_j,old_centre,how="inner",left_index=True)
new.head()

MergeError: Must pass right_on or right_index=True

In [ ]:
new = pd.merge(jvac,joined)
new.head()